# Scoring Script Test
The purpose of this test is a proof of concept implementation of NannyML performance estimation into the bridging scoring script. This could then be used to analyse data drift.

In [1]:
import pickle
import numpy as np
import re
import pandas as pd
import datetime as dt
import time
import matplotlib.pyplot as plt;
import warnings; warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from pom_NEW import *
import os
import sys
import gcsfs
sys.path.append(os.path.abspath("/home/jupyter/POM-feature-drift"))
from google.cloud import storage
from google.cloud import bigquery
import project_config as pc
import common_variables as cv

In [2]:
table_id = 'offer_bridging_eoo_base'
bucket_location = 'EU'
bucket_id       = 'gs://'+pc.bucket+'/pom_scoring'
file_name       = 'eoo_base_' + str(dt.datetime.now().date())
file_format     = 'CSV'
gcs_file_path   = os.path.join(bucket_id,file_name+'_*.csv')
prefix_name = 'pom_scoring'

In [3]:
#Connection to BQ
client = bigquery.Client(project=pc.project_id) #;
query = """ SELECT column_name, data_type
FROM """+pc.target_dataset+""".INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'offer_bridging_eoo_base'
AND data_type = 'DATE'
"""

date_cols = client.query(query).to_dataframe().iloc[:, 0].tolist()

In [4]:
#Load columns used in new models - speeds up process as we don't need to keep unused columns
cols = {'Account_Number', 'base_dt', 'eoo_base_obs_dt', 'Cohort', 'Rack_Rate', 'Ttl_Offer_Discount', 'Customer_Type', 'Country'}
for customer_type in ['DTV']:
    for country in ['UK']:
        for target_type in ['arpu', 'churn', 'ta']:
            for model_type in ['NT', 'L', 'M', 'H']:
                pickle_name = f'pickle_files/{customer_type}_{country}_{target_type}_{model_type}.pkl'
                if os.path.isfile(pickle_name):
                    with open(pickle_name, 'rb') as pickle_file:
                        model = pickle.load(pickle_file)
                    if (type(model) is XGBRegressor) or (type(model) is XGBClassifier):
                        model_columns = model.get_booster().feature_names
                    elif (type(model) is LGBMRegressor) or (type(model) is LGBMClassifier):
                        model_columns = model.feature_name_
                    elif (type(model) is CombinedModel):
                        model_columns = model.feature_name_
                    else:
                        print(f'MODEL TYPE NOT MATCHED {pickle_name}')
                        model_columns = []
                    if model_columns is None:
                        print(f'NONE COLUMNS {pickle_name}')
                        model_columns = []
                    cols = cols.union(model_columns)
rename_andrew = fix_columns()
cols = cols.union({x for x, y in rename_andrew.items() if y in cols})
cols = cols.union({x.rsplit('_', 1)[0] for x in cols})

In [5]:
#Load scoring data from GCS to pandas dataframe
df_main = read_pomdata_to_score(project = pc.project_id, bucket_name = pc.bucket, prefix_name = prefix_name)

blobs are ['pom-etl-process/pom_scoring/eoo_base_2022-12-05_000000000000.csv', 'pom-etl-process/pom_scoring/eoo_base_2022-12-05_000000000001.csv', 'pom-etl-process/pom_scoring/eoo_base_2022-12-05_000000000002.csv', 'pom-etl-process/pom_scoring/eoo_base_2022-12-05_000000000003.csv', 'pom-etl-process/pom_scoring/eoo_base_2022-12-05_000000000004.csv', 'pom-etl-process/pom_scoring/eoo_base_2022-12-05_000000000005.csv']
Total # of date variables are 167
Inital dataframe contains 456831 rows and 987 columns
Shape of the dataset is 456831 rows and 987 columns


## Clean EOO data

In [6]:
dpp = DataPreProcess(df=df_main.rename(columns={'EOO_Base_Obs_Dt' : 'eoo_base_obs_dt'}), cols=cols, date_cols=date_cols)

eoo_base_obs_dt available in the date df


In [7]:
del df_main

In [8]:
_ = dpp.process_dates()

In [9]:
_, _, _, _ = dpp.fill_missing()

In [11]:
_ = dpp.scale_numeric(excl_cols=['Account_Number', 'Movies_Active', 'Sports_Active', 'SGE_Active', 'HD_Active', 'MS_Active'])

In [12]:
_ = dpp.one_hot_encode(nunique=100)

[]


In [13]:
data_consolidated = dpp.concat_data(column_fix={'HD_Product_Holding_nan' : 'HD_Product_Holding_None'})
data_consolidated_scaled = dpp.concat_data(scale_numeric=True, column_fix=rename_andrew)

Data contains the total 456831 rows and 265 columns
Data contains the total 456831 rows and 265 columns


In [14]:
data_consolidated_uk_dtv = data_consolidated.loc[(data_consolidated['Country_UK'] == 1) & (data_consolidated['Customer_Type'] == 'DTV')]
data_consolidated_uk_dtv_scaled = data_consolidated_scaled.loc[(data_consolidated_scaled['Country_UK'] == 1) & (data_consolidated_scaled['Customer_Type'] == 'DTV')]

In [15]:
stepup_models = ['NT', 'L', 'M', 'H']

## Score TA Models

In [16]:
data_dict_uk_dtv_stepup = score_data(df=data_consolidated_uk_dtv_scaled, customer_type='DTV', country='UK', target='target_ta', model_types=stepup_models)

************************Model file DTV_UK_ta_NT exists************************
************************Scored data for model DTV_UK_ta_NT************************
************************Model file DTV_UK_ta_L exists************************
************************Scored data for model DTV_UK_ta_L************************
************************Model file DTV_UK_ta_M exists************************
************************Scored data for model DTV_UK_ta_M************************
************************Model file DTV_UK_ta_H exists************************
************************Scored data for model DTV_UK_ta_H************************


In [17]:
scored_df_ta_uk_dtv_new = pd.concat(list(data_dict_uk_dtv_stepup.values()), axis=1)
scored_df_ta = pd.concat([scored_df_ta_uk_dtv_new])

In [18]:
del scored_df_ta_uk_dtv_new

In [19]:
data_consolidated_uk_dtv_scaled.head()

,Lima_ICD_Flag,Account_Number,Rack_Rate,Offer_Discount_SoD,Ttl_Offer_Discount,ARPU,Prev_Offer_Amount_LR,Curr_Offer_Amount_LR,Curr_Offer_Amount_HD,Offers_Applied_Lst_36M_HD,Curr_Offer_Amount_HD_Pack,Curr_Offer_Amount_SKY_BOX_SETS,Curr_Offer_Amount_SKY_KIDS,Contract_Offers_Applied_Lst_12M_DTV,Curr_Contract_Offer_Amount_BB,Curr_Promo_Offer_Amount_DTV,DTV_Activations_In_Last_3Yr,BB_Subscriber_Activations_In_Last_3Yr,BB_Churns_In_Last_3Yr,MS_Active,HD_Active,SGE_Active,TAs_in_last_24m,Age,Prev_Offer_Amount_DTV,Curr_Offer_Amount_DTV,Offers_Applied_Lst_24M_DTV,Prev_Offer_Amount_BB,Prev_Offer_Amount_Movies,Curr_Offer_Amount_Movies,Sports_Active,Sports_Product_Count,Movies_Active,OD_DLs_Completed_In_Last_7d,OD_DLs_Completed_In_Last_30d,OD_DLs_Completed_In_Last_60d,Sky_Consumer_Market_Share,Virgin_Consumer_Market_Share,Throughput_Speed,max_speed_uplift,cancels_3mth,cancels_6mth,cancels_9mth,cancels_12mth,mobile_3mth,mobile_6mth,mobile_9mth,bill_6mth,signin_6mth,vip_6mth,vip_12mth,help_6mth,total_viewing_duration_overlap_3m,total_viewing_duration_overlap_2m,total_viewing_duration_overlap_1m,linear_viewing_total_overlap_1m,linear_viewing_total_overlap_3m,Last_Offer_Applied_Dt_HD_monthdiff,Last_Contract_Offer_Applied_Dt_DTV_monthdiff,Last_Promo_Offer_Applied_Dt_BB_monthdiff,DTV_Last_Activation_Dt_monthdiff,BB_Last_Activation_Dt_monthdiff,BB_Last_Churn_Dt_monthdiff,DTV_Last_PC_Effective_To_Dt_monthdiff,DTV_Last_Active_Block_Dt_monthdiff,DTV_Last_AB_Effective_To_Dt_monthdiff,last_TA_dt_monthdiff,Last_Value_Call_Dt_monthdiff,Last_Platform_Retention_Call_Dt_monthdiff,Last_Service_Call_Dt_monthdiff,Last_All_Call_Dt_monthdiff,Curr_Offer_Start_Dt_DTV_monthdiff,Last_Offer_Applied_Dt_DTV_monthdiff,Last_Offer_Applied_Dt_BB_monthdiff,Last_Completed_OD_DL_Dt_monthdiff,Last_Credit_Dt_monthdiff,Customer_Type,BB_Product_Holding_12GB,BB_Product_Holding_BroadbandEssential20ADSL,BB_Product_Holding_BroadbandSuperfast40FTTC,BB_Product_Holding_BroadbandSuperfast80FTTC,BB_Product_Holding_Connect,BB_Product_Holding_ConnectLiteROILegacy,BB_Product_Holding_ConnectUnlimitedROILegacy,BB_Product_Holding_ConnectUnlimitedROI,BB_Product_Holding_EssentialROI,BB_Product_Holding_EssentialPlus,BB_Product_Holding_Everyday,BB_Product_Holding_FibreROILegacy,BB_Product_Holding_FibreLite,BB_Product_Holding_FibreMax,BB_Product_Holding_FibreUltra350Mb,BB_Product_Holding_FibreUnlimitedROILegacy,BB_Product_Holding_FibreUnlimitedPlus,BB_Product_Holding_FibreUnlimitedPro,BB_Product_Holding_FibreUnlimitedProFTTC,BB_Product_Holding_FibreUnlimitedProFTTP,BB_Product_Holding_GigafastFTTP,BB_Product_Holding_GigafastROI,BB_Product_Holding_SKYFibreMaxSOGEA,BB_Product_Holding_SKYFibreUnlimitedSOGEA,BB_Product_Holding_SkyBroadbandLite,BB_Product_Holding_SkyBroadbandLiteROILegacy,BB_Product_Holding_SkyFibre,BB_Product_Holding_SkySuperfastSFTC,BB_Product_Holding_SkyUltrafastMaxROI,BB_Product_Holding_SkyUltrafastPlusROI,BB_Product_Holding_SuperfastFTTP,BB_Product_Holding_SuperfastROI,BB_Product_Holding_Superfast35FTTC,BB_Product_Holding_Superfast35FTTP,BB_Product_Holding_Superfast35SOGEA,BB_Product_Holding_UltrafastROI,BB_Product_Holding_Ultrafast1FTTP,BB_Product_Holding_Ultrafast160SOGfast,BB_Product_Holding_UltrafastPlusFTTP,BB_Product_Holding_UltrafastPlusROI,BB_Product_Holding_Unlimited,BB_Product_Holding_UnlimitedLegacy,BB_Product_Holding_UnlimitedROILegacy,BB_Product_Holding_UnlimitedFibre,BB_Product_Holding_UnlimitedPro,BB_Product_Holding_nan,Talk_Product_Holding_Anytime,Talk_Product_Holding_AnytimeROI,Talk_Product_Holding_Freetime,Talk_Product_Holding_FreetimeROI,Talk_Product_Holding_Other,Talk_Product_Holding_SkyPayAsYouTalk,Talk_Product_Holding_SkyTalkAnytimeExtra,Talk_Product_Holding_SkyTalkAnytimeExtra247,Talk_Product_Holding_SkyTalkAnytimeExtraAnytimeUK,Talk_Product_Holding_SkyTalkAnytimeExtraAnytime,Talk_Product_Holding_SkyTalkAnytimeExtraVoIP,Talk_Product_Holding_SkyTalkEveningsandWeekendsExtra,Talk_Product_Holding_SkyTalkInternationalExtra,Talk_Product_Holding_Weekend,Talk_Product_

## NannyML Performance Estimation Implementation

In [21]:
ta_data = data_consolidated_uk_dtv_scaled.join(scored_df_ta, on = 'Account_Number')

In [22]:
ta_data.head()

,Lima_ICD_Flag,Account_Number,Rack_Rate,Offer_Discount_SoD,Ttl_Offer_Discount,ARPU,Prev_Offer_Amount_LR,Curr_Offer_Amount_LR,Curr_Offer_Amount_HD,Offers_Applied_Lst_36M_HD,Curr_Offer_Amount_HD_Pack,Curr_Offer_Amount_SKY_BOX_SETS,Curr_Offer_Amount_SKY_KIDS,Contract_Offers_Applied_Lst_12M_DTV,Curr_Contract_Offer_Amount_BB,Curr_Promo_Offer_Amount_DTV,DTV_Activations_In_Last_3Yr,BB_Subscriber_Activations_In_Last_3Yr,BB_Churns_In_Last_3Yr,MS_Active,HD_Active,SGE_Active,TAs_in_last_24m,Age,Prev_Offer_Amount_DTV,Curr_Offer_Amount_DTV,Offers_Applied_Lst_24M_DTV,Prev_Offer_Amount_BB,Prev_Offer_Amount_Movies,Curr_Offer_Amount_Movies,Sports_Active,Sports_Product_Count,Movies_Active,OD_DLs_Completed_In_Last_7d,OD_DLs_Completed_In_Last_30d,OD_DLs_Completed_In_Last_60d,Sky_Consumer_Market_Share,Virgin_Consumer_Market_Share,Throughput_Speed,max_speed_uplift,cancels_3mth,cancels_6mth,cancels_9mth,cancels_12mth,mobile_3mth,mobile_6mth,mobile_9mth,bill_6mth,signin_6mth,vip_6mth,vip_12mth,help_6mth,total_viewing_duration_overlap_3m,total_viewing_duration_overlap_2m,total_viewing_duration_overlap_1m,linear_viewing_total_overlap_1m,linear_viewing_total_overlap_3m,Last_Offer_Applied_Dt_HD_monthdiff,Last_Contract_Offer_Applied_Dt_DTV_monthdiff,Last_Promo_Offer_Applied_Dt_BB_monthdiff,DTV_Last_Activation_Dt_monthdiff,BB_Last_Activation_Dt_monthdiff,BB_Last_Churn_Dt_monthdiff,DTV_Last_PC_Effective_To_Dt_monthdiff,DTV_Last_Active_Block_Dt_monthdiff,DTV_Last_AB_Effective_To_Dt_monthdiff,last_TA_dt_monthdiff,Last_Value_Call_Dt_monthdiff,Last_Platform_Retention_Call_Dt_monthdiff,Last_Service_Call_Dt_monthdiff,Last_All_Call_Dt_monthdiff,Curr_Offer_Start_Dt_DTV_monthdiff,Last_Offer_Applied_Dt_DTV_monthdiff,Last_Offer_Applied_Dt_BB_monthdiff,Last_Completed_OD_DL_Dt_monthdiff,Last_Credit_Dt_monthdiff,Customer_Type,BB_Product_Holding_12GB,BB_Product_Holding_BroadbandEssential20ADSL,BB_Product_Holding_BroadbandSuperfast40FTTC,BB_Product_Holding_BroadbandSuperfast80FTTC,BB_Product_Holding_Connect,BB_Product_Holding_ConnectLiteROILegacy,BB_Product_Holding_ConnectUnlimitedROILegacy,BB_Product_Holding_ConnectUnlimitedROI,BB_Product_Holding_EssentialROI,BB_Product_Holding_EssentialPlus,BB_Product_Holding_Everyday,BB_Product_Holding_FibreROILegacy,BB_Product_Holding_FibreLite,BB_Product_Holding_FibreMax,BB_Product_Holding_FibreUltra350Mb,BB_Product_Holding_FibreUnlimitedROILegacy,BB_Product_Holding_FibreUnlimitedPlus,BB_Product_Holding_FibreUnlimitedPro,BB_Product_Holding_FibreUnlimitedProFTTC,BB_Product_Holding_FibreUnlimitedProFTTP,BB_Product_Holding_GigafastFTTP,BB_Product_Holding_GigafastROI,BB_Product_Holding_SKYFibreMaxSOGEA,BB_Product_Holding_SKYFibreUnlimitedSOGEA,BB_Product_Holding_SkyBroadbandLite,BB_Product_Holding_SkyBroadbandLiteROILegacy,BB_Product_Holding_SkyFibre,BB_Product_Holding_SkySuperfastSFTC,BB_Product_Holding_SkyUltrafastMaxROI,BB_Product_Holding_SkyUltrafastPlusROI,BB_Product_Holding_SuperfastFTTP,BB_Product_Holding_SuperfastROI,BB_Product_Holding_Superfast35FTTC,BB_Product_Holding_Superfast35FTTP,BB_Product_Holding_Superfast35SOGEA,BB_Product_Holding_UltrafastROI,BB_Product_Holding_Ultrafast1FTTP,BB_Product_Holding_Ultrafast160SOGfast,BB_Product_Holding_UltrafastPlusFTTP,BB_Product_Holding_UltrafastPlusROI,BB_Product_Holding_Unlimited,BB_Product_Holding_UnlimitedLegacy,BB_Product_Holding_UnlimitedROILegacy,BB_Product_Holding_UnlimitedFibre,BB_Product_Holding_UnlimitedPro,BB_Product_Holding_nan,Talk_Product_Holding_Anytime,Talk_Product_Holding_AnytimeROI,Talk_Product_Holding_Freetime,Talk_Product_Holding_FreetimeROI,Talk_Product_Holding_Other,Talk_Product_Holding_SkyPayAsYouTalk,Talk_Product_Holding_SkyTalkAnytimeExtra,Talk_Product_Holding_SkyTalkAnytimeExtra247,Talk_Product_Holding_SkyTalkAnytimeExtraAnytimeUK,Talk_Product_Holding_SkyTalkAnytimeExtraAnytime,Talk_Product_Holding_SkyTalkAnytimeExtraVoIP,Talk_Product_Holding_SkyTalkEveningsandWeekendsExtra,Talk_Product_Holding_SkyTalkInternationalExtra,Talk_Product_Holding_Weekend,Talk_Product_